## We Apply Decision Tree which causes overfitting then we apply random forest with gridsearch cv which makes better accuracy 

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import  GridSearchCV


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Read the CSV file
df = pd.read_csv('t20.csv')

# Check for NaN values and handle them
df = df.dropna()

# Initialize LabelEncoders
label_encoder_team = LabelEncoder()
label_encoder_venue = LabelEncoder()
label_encoder_winner = LabelEncoder()

# Encode teams
all_teams = pd.concat([df['Bat First'], df['Bat Second']])
label_encoder_team.fit(all_teams)
df['team1_encoded'] = label_encoder_team.transform(df['Bat First'])
df['team2_encoded'] = label_encoder_team.transform(df['Bat Second'])

# Encode venue
df['venue_encoded'] = label_encoder_venue.fit_transform(df['Venue'])

# Encode winner
df['winner_encoded'] = label_encoder_winner.fit_transform(df['Winner'])

# Features and target
X = df[['team1_encoded', 'team2_encoded', 'venue_encoded']]
y = df['winner_encoded']


In [7]:
# Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Perform grid search with cross-validation
rf_clf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf_clf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Best parameters from the grid search
best_params = grid_search.best_params_
print(f"Best parameters found: {best_params}")

# Train the Random Forest classifier with the best parameters
best_rf_clf = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_rf_clf.predict(X_test)


Fitting 5 folds for each of 216 candidates, totalling 1080 fits


c:\Users\Hanzalah\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\Hanzalah\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best parameters found: {'bootstrap': True, 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}


In [8]:

from sklearn.metrics import accuracy_score
# Make predictions on training and test sets
y_train_pred = best_rf_clf.predict(X_train)
y_test_pred = best_rf_clf.predict(X_test)

# Calculate accuracy scores
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Train Accuracy: {train_accuracy:.2f}")
print(f"Test Accuracy: {test_accuracy:.2f}")

Train Accuracy: 0.90
Test Accuracy: 0.50


### our accuracy increases by using random forest

In [ ]:
import joblib

# Save the model and label encoders
joblib.dump(model, 'rf_t20_model.pkl')
joblib.dump(label_encoders, 't20_label_encoders.pkl')